In [7]:
import os
import sys

spark_path = "/Users/weizhong/stk/spark-1.6.0-bin-hadoop2.6"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path
#os.environ['PYSPARK_PYTHON'] = 'C:\Users\Wei Zhong\Anaconda3\envs\python2\python' # <---

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.9-src.zip")

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

sc = SparkContext("local")
sqlCtx = SQLContext(sc)

In [9]:
# Generate Random Data
import itertools
import random
students = ['John', 'Mike','Matt']
subjects = ['Math', 'Sci', 'Geography', 'History']
random.seed(1)
data = []
 
for (student, subject) in itertools.product(students, subjects):
    data.append((student, subject, random.randint(0, 100)))
 
# Create Schema Object
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
            StructField("student", StringType(), nullable=False),
            StructField("subject", StringType(), nullable=False),
            StructField("score", IntegerType(), nullable=False)
    ])
 
# Create DataFrame 
#from pyspark.sql import HiveContext
#sqlContext = HiveContext(sc)
rdd = sc.parallelize(data)
df = sqlCtx.createDataFrame(rdd, schema)
 
# Define udf
from pyspark.sql.functions import udf
def scoreToCategory(score):
    if score >= 80: return 'A'
    elif score >= 60: return 'B'
    elif score >= 35: return 'C'
    else: return 'D'
 
udfScoreToCategory=udf(scoreToCategory, StringType())
df.withColumn("category", udfScoreToCategory("score")).show(10)

+-------+---------+-----+--------+
|student|  subject|score|category|
+-------+---------+-----+--------+
|   John|     Math|   13|       D|
|   John|      Sci|   85|       A|
|   John|Geography|   77|       B|
|   John|  History|   25|       D|
|   Mike|     Math|   50|       C|
|   Mike|      Sci|   45|       C|
|   Mike|Geography|   65|       B|
|   Mike|  History|   79|       B|
|   Matt|     Math|    9|       D|
|   Matt|      Sci|    2|       D|
+-------+---------+-----+--------+
only showing top 10 rows



In [10]:
sc.stop()